In [51]:
from utils import *
from configs import *
import pandas as pd
import numpy as np

In [9]:
with open('/workspace/Indic-SemParse/Indic-SemParse/filtered_data/itop/hi.json', 'r') as f:
    data = json.load(f)

    
train = data['train']

train_df = pd.DataFrame(train)

In [10]:
lf = train_df['logical_form'].to_list()

In [11]:
seq2seq_models

['ai4bharat/IndicBART', 'google/mt5-base', 'facebook/mbart-large-50']

In [12]:
tokenizer = get_tokenizer('ai4bharat/IndicBART', "itop","en")

In [19]:
tks = [len(tokenizer.tokenize(l)) for l in lf]

In [28]:
sum([tk > 64 for tk in tks])

2

In [29]:
with open('/workspace/Indic-SemParse/Indic-SemParse/filtered_data/indic-TOP/hi.json', 'r') as f:
    data = json.load(f)

    
train = data['train']

train_df = pd.DataFrame(train)

In [39]:
lf = train_df['decoupled logical form'].to_list()

In [40]:
tokenizer = get_tokenizer('ai4bharat/IndicBART', "indic-TOP","en")
tks = [len(tokenizer.tokenize(l)) for l in lf]

sum([tk > 64 for tk in tks])

Token indices sequence length is longer than the specified maximum sequence length for this model (67 > 64). Running this sequence through the model will result in indexing errors


1

In [42]:
with open('/workspace/Indic-SemParse/Indic-SemParse/filtered_data/indic-atis/hi.json', 'r') as f:
    data = json.load(f)

    
train = data['train']

train_df = pd.DataFrame(train)

lf = train_df['decoupled logical form'].to_list()

tokenizer = get_tokenizer('ai4bharat/IndicBART', "indic-atis","en")
tks = [len(tokenizer.tokenize(l)) for l in lf]

sum([tk > 64 for tk in tks])

Token indices sequence length is longer than the specified maximum sequence length for this model (35 > 32). Running this sequence through the model will result in indexing errors


7

In [43]:
ex = "[IN:CREATE_REMINDER [SL:PERSON_REMINDED me ] [SL:TODO pay my flowers order ] [SL:DATE_TIME tomorrow morning ] ] [SL:DATE_TIME tomorrow morning ] ] [SL:TODO pay my flowers order ] ] [SL:DATE_TIME tomorrow morning ] ]"

In [46]:
def post_process_lf(sent):
    stck = []
    res = []
    
    toks = sent.split()
    
    
    for tok in toks:
        res.append(tok)
        if tok.startswith("["):
            stck.append("[")
        elif tok == ']':
            stck.pop()
        
        if len(stck) == 0:
            break
        
        
    return ' '.join(res) 

In [47]:
post_process_lf(ex)

'[IN:CREATE_REMINDER [SL:PERSON_REMINDED me ] [SL:TODO pay my flowers order ] [SL:DATE_TIME tomorrow morning ] ]'

In [50]:
with open("/workspace/Indic-SemParse/results/indic_train/itop/IndicBART/hi.json", "r") as f:
    data = json.load(f)
    data = data['outputs']

In [53]:
df = pd.DataFrame(data)

In [57]:
df['predictions'] = list(map(post_process_lf, df['predictions'].to_list()))

In [61]:
df

,idx,intent,spans,question,domain,lang,logical_form,tokenized_question,processed question,translated question,postprocessed_translated_question,backtranslated_post_processed_questions,orig,src,trg,predictions,labels,exact_match,tree_labelled_f1,levenshtein_distance
0,3231353336363636,IN:UPDATE_METHOD_CALL,,can you enable video call,calling,en_XX,[IN:UPDATE_METHOD_CALL ],"{""tokens"":[""can"",""you"",""enable"",""video"",""call""...",can you enable video call,क्या आप वीडियो कॉल कर सकते हैं?,क्या आप विडियो कॉल कर सकते हैं?,Can you make a video call?,can you enable video call,क्या आप विडियो कॉल कर सकते हैं?,[IN:UPDATE_METHOD_CALL ],[IN:CREATE_CALL ],[IN:UPDATE_METHOD_CALL ],0.0,0.0,0.293103
1,3231323239313230,IN:CREATE_REMINDER,"7:9:SL:PERSON_REMINDED,13:36:SL:TODO,37:53:SL:...",Remind me to pay for my flower order tomorrow ...,reminder,en_XX,[IN:CREATE_REMINDER [SL:PERSON_REMINDED me ] [...,"{""tokens"":[""Remind"",""me"",""to"",""pay"",""for"",""my""...",Remind me to pay for my flower order tomorrow ...,मुझे कल सुबह अपने फूल ऑर्डर के लिए भुगतान करने...,मुझे कल मॉर्निंग अपने फूल ऑर्डर के लिए भुगतान ...,Remind me to pay for your flower order tomorro...,Remind me to pay for my flower order tomorrow ...,मुझे कल मॉर्निंग अपने फूल ऑर्डर के लिए भुगतान ...,[IN:CREATE_REMINDER [SL:PERSON_REMINDED me ] [...,[IN:CREATE_REMINDER [SL:PERSON_REMINDED me ] [...,[IN:CREATE_REMINDER [SL:PERSON_REMINDED me ] [...,0.0,0.0,0.693009
2,3231333532393638,IN:GET_AVAILABILITY,3:6:SL:CONTACT,Is Kim online?,calling,en_XX,[IN:GET_AVAILABILITY [SL:CONTACT Kim ] ],"{""tokens"":[""Is"",""Kim"",""online"",""?""],""tokenSpan...",Is Kim online ?,क्या किम ऑनलाइन हैं?,क्या किम ऑनलाइन हैं?,Is Kim online?,Is Kim online?,क्या किम ऑनलाइन हैं?,[IN:GET_AVAILABILITY [SL:CONTACT Kim ] ],[IN:GET_AVAILABILITY [SL:CONTACT Kim ] ],[IN:GET_AVAILABILITY [SL:CONTACT Kim ] ],0.0,0.0,0.536913
3,3231303135353437,IN:CREATE_CALL,24:28:SL:CONTACT,Start a video call with Phil.,calling,en_XX,[IN:CREATE_CALL [SL:CONTACT Phil ] ],"{""tokens"":[""Start"",""a"",""video"",""call"",""with"",""...",Start a video call with Phil .,फिल के साथ एक वीडियो कॉल शुरू करें।,फिल के साथ एक विडियो कॉल शुरू करें।,Start a video call with Phil.,Start a video call with Phil.,फिल के साथ एक विडियो कॉल शुरू करें।,[IN:CREATE_CALL [SL:CONTACT Phil ] ],[IN:CREATE_CALL [SL:CONTACT Phil ] ],[IN:CREATE_CALL [SL:CONTACT Phil ] ],0.0,0.0,0.413793
4,39353535393234,IN:CREATE_ALARM,13:35:SL:DATE_TIME,Set an alarm for 7 minutes from now,alarm,en_XX,[IN:CREATE_ALARM [SL:DATE_TIME for 7 minutes f...,"{""tokens"":[""Set"",""an"",""alarm"",""for"",""7"",""minut...",Set an alarm for seven minutes from now,अब से सात मिनट के लिए अलार्म सेट करें,अब से सात मिनट के लिए अलार्म सेट करें,Set the alarm for seven minutes from now,Set an alarm for 7 minutes from now,अब से सात मिनट के लिए अलार्म सेट करें,[IN:CREATE_ALARM [SL:DATE_TIME for 7 minutes f...,[IN:CREATE_ALARM [SL:DATE_TIME for 7 minutes f...,[IN:CREATE_ALARM [SL:DATE_TIME for 7 minutes f...,0.0,0.4,0.721519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,39353636383532,IN:GET_WEATHER,18:26:SL:WEATHER_ATTRIBUTE,Should I bring an umbrella?,weather,en_XX,[IN:GET_WEATHER [SL:WEATHER_ATTRIBUTE umbrella...,"{""tokens"":[""Should"",""I"",""bring"",""an"",""umbrella...",Should I bring an umbrella ?,क्या मुझे एक छतरी लानी चाहिए?,क्या मुझे एक छतरी लानी चाहिए?,Should I bring an umbrella?,Should I bring an umbrella?,क्या मुझे एक छतरी लानी चाहिए?,[IN:GET_WEATHER [SL:WEATHER_ATTRIBUTE umbrella...,[IN:GET_WEATHER [SL:WEATHER_ATTRIBUTE umbrella...,[IN:GET_WEATHER [SL:WEATHER_ATTRIBUTE umbrella...,0.0,0.0,0.588235
499,3231353437363232,IN:GET_AVAILABILITY,19:23:SL:CONTACT,Can you tell me if Luci is online now,calling,en_XX,[IN:GET_AVAILABILITY [SL:CONTACT Luci ] ],"{""tokens"":[""Can"",""you"",""tell"",""me"",""if"",""Luci""...",Can you tell me if Luci is online now,क्या आप मुझे बता सकते हैं कि लूसी अब ऑनलाइन है?,क्या आप मुझे बता सकते हैं कि लूसी अब ऑनलाइन है

In [64]:
rows = df.to_dict("records")

In [66]:
with open("/workspace/Indic-SemParse/results/indic_train/itop/IndicBART/hi.json", "w", encoding="utf-8") as f:
    json.dump({"outputs": rows}, f, indent=6, ensure_ascii=False)
